In [3]:
import pandas as pd
from IPython.display import display, HTML
import numpy as np
from hydra import initialize, compose

from assembly_tools import load_experimental_grid, format_table, extract_raw_performance

In [2]:
%load_ext autoreload
%autoreload 2

### Assembly of experimental results 2


In [45]:
# load default params. They can be changed however you want.
with initialize(version_base=None, config_path="config"):
    cfg = compose(config_name='extract.yaml')
    print(cfg)
general_params =  ["resolution_value", "ds_name", "method_hps.max_lag"]
hp_stack = []
scoring = 'Individual AUROC'
cfg.ds_order = ["random_5"]

{'metrics': ['Max Acc thresh', 'Max Acc', 'Max individual Acc thresh', 'Max individual Acc', 'Null Acc', 'Max F1 thresh', 'Max F1', 'Max individual F1 thresh', 'Max individual F1', 'Null F1', 'AUROC', 'Individual AUROC', 'Null AUROC', 'runtime'], 'hp_list': ['data_preprocess.resolution', 'data_preprocess.normalize', 'label_path', 'method.max_lag', 'method.var_absolute_values', 'method.filter_mode', 'method.architecture'], 'rename': {'resolution_value': 'data_preprocess.resolution', 'method_hps.var_absolute_values': 'method.var_absolute_values', 'method_hps.filter_mode': 'method.filter_mode', 'method_hps.max_lag': 'method.max_lag', 'method_hps.cp_architecture': 'method.architecture', 'ds_name': 'label_path', 'normalize': 'data_preprocess.normalize', 'window_data_month_value': 'data_preprocess.subset_month', 'window_data_year_value': 'data_preprocess.subset_year'}, 'ds_order': ['close_3', 'close_5', 'root_cause_3', 'root_cause_5', '1_random_3', '1_random_5', 'confounder_3', 'confounder_5

In [58]:
performance_stack = []

for name in ["reverse", "cross", "combo", "var_", "varlingam","pcmci","dyno","cp"]:
    data = pd.read_csv("grid_export_exp2/" + name + ".csv", index_col=0)
    res = ["data_path",  "data_preprocess.subset_year"]
    formatted = format_table(data,cfg, scoring= scoring, restriction=res)
    lines = extract_raw_performance(formatted,cfg,scoring,name="", restriction=res)
    lines.columns = [name]
    hp_stack.append(formatted)
    performance_stack.append(lines)
    
out = pd.concat(performance_stack,axis=1).T  
out = out[[out.columns[1], out.columns[0], out.columns[2]]]
out.columns = ["Full TS", "No Rain", "Flood"]


out.index = ["RP", "CC", "RPCC", "VAR", "Varlingam", "PCMCI", "Dynotears", "CP"]
out = out.astype(float).round(3)
style = out.style.highlight_max(axis=1,  props='color:black;font-weight:bold')
style = style.format("{:.2f}")
latex_str = style.to_latex(convert_css=True)
display(HTML(style.to_html()))
print(latex_str.replace("nan", "\\textdagger"))

Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')


,Full TS,No Rain,Flood
RP,0.78,0.78,0.70
CC,0.81,0.74,0.80
RPCC,0.81,0.79,0.74
VAR,0.85,0.86,0.83
Varlingam,0.72,0.67,0.74
PCMCI,0.60,0.69,0.60
Dynotears,0.79,0.60,0.80
CP,0.65,0.66,0.74


\begin{tabular}{lrrr}
 & Full TS & No Rain & Flood \\
RP & \color{black} \bfseries 0.78 & 0.78 & 0.70 \\
CC & \color{black} \bfseries 0.81 & 0.74 & 0.80 \\
RPCC & \color{black} \bfseries 0.81 & 0.79 & 0.74 \\
VAR & 0.85 & \color{black} \bfseries 0.86 & 0.83 \\
Varlingam & 0.72 & 0.67 & \color{black} \bfseries 0.74 \\
PCMCI & 0.60 & \color{black} \bfseries 0.69 & 0.60 \\
Dynotears & 0.79 & 0.60 & \color{black} \bfseries 0.80 \\
CP & 0.65 & 0.66 & \color{black} \bfseries 0.74 \\
\end{tabular}



In [74]:
# As we run CDMI distributed and in a different format, we exclude it here as the raw files are too big. 
# Further it is not trainable on the small regimes making it not comparable.
# For documentation purposes: This is how we included it.

performance_stack = []

for name in ["reverse", "cross", "combo", "var_", "varlingam","pcmci","dyno","cp"]:
    data = pd.read_csv("grid_export_exp2/" + name + ".csv", index_col=0)
    res = ["data_path",  "data_preprocess.subset_year"]
    formatted = format_table(data,cfg, scoring= scoring, restriction=res)
    lines = extract_raw_performance(formatted,cfg,scoring,name="", restriction=res)
    lines.columns = [name]
    hp_stack.append(formatted)
    performance_stack.append(lines)

### CDMI
data = load_experimental_grid(mypath=cfg.data_path + "exp2/", method_name="cdmi").T
out = pd.concat(performance_stack,axis=1).T
out = out[[out.columns[1], out.columns[0], out.columns[2]]]
out.columns = ["Full TS", "No Rain", "Flood"]
out.loc["cdmi"] = [data["Individual AUROC"].mean(), np.nan, np.nan]
out.index = ["RP", "CC", "RPCC", "VAR", "Varlingam", "PCMCI", "Dynotears", "CP", "CDMI"]
# switch order
out =out.loc[list(out.index[:-2]) + ["CDMI"] + ["CP"]]
### CDMI
style = out.style.highlight_max(axis=1,  props='color:black;font-weight:bold')
style = style.format("{:.2f}")
latex_str = style.to_latex(convert_css=True)
display(HTML(style.to_html()))
print(latex_str.replace("nan", "\\textdagger"))

Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')
Fail: ('resources/rivers_flood.csv', '2021', 'random_5')


,Full TS,No Rain,Flood
RP,0.78,0.78,0.70
CC,0.81,0.74,0.80
RPCC,0.81,0.79,0.75
VAR,0.85,0.86,0.83
Varlingam,0.72,0.67,0.74
PCMCI,0.61,0.69,0.60
Dynotears,0.79,0.60,0.80
CDMI,0.83,nan,nan
CP,0.65,0.66,0.74


\begin{tabular}{llll}
 & Full TS & No Rain & Flood \\
RP & \color{black} \bfseries 0.78 & 0.78 & 0.70 \\
CC & \color{black} \bfseries 0.81 & 0.74 & 0.80 \\
RPCC & \color{black} \bfseries 0.81 & 0.79 & 0.75 \\
VAR & 0.85 & \color{black} \bfseries 0.86 & 0.83 \\
Varlingam & 0.72 & 0.67 & \color{black} \bfseries 0.74 \\
PCMCI & 0.61 & \color{black} \bfseries 0.69 & 0.60 \\
Dynotears & 0.79 & 0.60 & \color{black} \bfseries 0.80 \\
CDMI & \color{black} \bfseries 0.83 & \textdagger & \textdagger \\
CP & 0.65 & 0.66 & \color{black} \bfseries 0.74 \\
\end{tabular}



### CHeck HPS for control

In [5]:
data = pd.read_csv("grid_export_exp2/" + "var_" + ".csv", index_col=0)


In [15]:
data = data[data["data_path"] == "resources/rivers_flood.csv"]

In [17]:
data.sort_values("Individual AUROC",ascending=False).iloc[0] # THis is the best performance of a VAR model for Exp2. 

Max Acc thresh                                    3.058926
Max Acc                                           0.815171
Max individual Acc thresh                          4.62454
Max individual Acc                                0.860542
Null Acc                                          0.799813
Max F1 thresh                                    48.696238
Max F1                                            0.568949
Max individual F1 thresh                         17.070496
Max individual F1                                 0.676528
Null F1                                           0.333593
AUROC                                             0.831272
Individual AUROC                                  0.834518
Null AUROC                                             0.5
runtime                             0 days 00:26:19.201219
data_preprocess.resolution                           15min
data_preprocess.normalize                            False
label_path                                        random